In [1]:
from sklearn.datasets import load_iris  
import numpy as np
from IrisFisher import softmax, softmax_grad, logloss, sigmoid

In [23]:
data = load_iris()

def UnisonShuffle(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

def random_init(size, left = -0.1, right = 0.1):
    return np.random.random(size = size) * (right + left) + left

def net_init_1(params):
    params["L0"] = np.zeros((1, 4))

    params["W0"] = random_init((4, 4))
    params['b0'] = random_init((1, 4))

    params["L1"] = np.zeros((1, 4))
    params["P1"] = np.zeros((1, 4))
    
    params["W1"] = random_init((4, 3))
    params['b1'] = random_init((1, 3))
    
    params["L2"] = np.zeros((1, 3))
    params["P2"] = np.zeros((1, 3))
    
    params["E"] = np.zeros((1,1))
    
def check_shape_1(X, y):
    if X.shape !=  (1, 4):
        raise ValueError("X : Expect shape (1, 4), given {0}".format(X.shape))
    if y.shape !=  (1, 3):
        raise ValueError("X : Expect shape (1, 3), given {0}".format(y.shape))
    

In [24]:
def forward_prop_1(X, y, params):
    check_shape_1(X, y)
    
    params["L0"] = X

    params["L1"] = np.dot(params["L0"], params["W0"]) + params["b0"]
    params["P1"] = sigmoid(params["L1"])
    
    # params["P1"] = X
    
    params["L2"] = np.dot(params["P1"], params["W1"]) + params["b1"]
    params["P2"] = softmax(params["L2"])
    
    params["E"] = logloss(params["P2"], y)
    
def back_prop_1(X, y, params, giper_params):
    check_shape_1(X, y)
    
    params["dE/dP2"] = - y / params["P2"] # 1x3
    params["dP2/dL2"] = softmax_grad(params["P2"]) # 3x3
    params["dE/dL2"] = np.dot(params["dE/dP2"], params["dP2/dL2"]) # 1x3
    params["dE/dW1"] = np.dot(params["P1"].T, params["dE/dL2"]) #4x3
    params["dE/db1"] = params["dE/dL2"]
    
    params["dE/dP1"] = np.dot(params["dE/dL2"], params["W1"].T)# 1x4
    params["dP1/dL1"] = params["P1"] * (1 - params["P1"]) #1x4
    params["dE/dL1"] = params["dE/dP1"] * params["dP1/dL1"] # 1x4
    # params["dE/dL1"] = np.dot(params["dE/dL2"], params["W1"].T)
    params["dE/dW0"] = np.dot(params["L0"].T, params["dE/dL1"]) # 4x4
    params["dE/db0"] = params["dE/dL1"]
    
    params["W1"] -= giper_params["learning rate"] * params["dE/dW1"]
    params["b1"] -= giper_params["learning rate"] * params["dE/db1"]
    
    params["W0"] -= giper_params["learning rate"] * params["dE/dW0"]
    params["b0"] -= giper_params["learning rate"] * params["dE/db0"]
    

In [33]:
X = data["data"]
y_lables = data["target"]
y = np.zeros((len(y_lables), 3))
for i, a in enumerate(y_lables):
    y[i][a] = 1

np.random.seed(1)
test = np.random.choice(149, 20, replace=False)
print(test)
X, y = UnisonShuffle(X, y)
X = X.reshape(-1, 1, 4)
y = y.reshape(-1, 1, 3)
X_test = X[test]
y_test = y[test]
X = X[~test]
y = y[~test]


params = dict()
giper_params = dict()
giper_params["learning rate"] = 0.01
lenX = len(y)


net_init_1(params)
    

[145  89  54  77  84 105  91  53  44  59   5  16  93  14  58  33  73  29
  66  35]


In [34]:
#TODO make normalization 

for i in range(lenX * 1000):
    X_cur = X[i % lenX]
    y_cur = y[i % lenX]
    forward_prop_1(X_cur, y_cur, params)
    back_prop_1(X_cur, y_cur, params, giper_params)
    if (i % lenX) == lenX - 1:
        error_counter = 0
        for iter, item in enumerate(y_test):
            X_cur = X_test[iter]
            y_cur = y_test[iter]
            forward_prop_1(X_cur, y_cur, params)
            if np.argmax(y_cur) != np.argmax(params["P2"]):
                error_counter += 1
            # print(np.argmax(y_cur), np.argmax(params["P2"]))
            # print(params["P2"])
        print(error_counter/20, end = "  ")
        
        error_counter = 0
        for iter, item in enumerate(y):
            X_cur = X[iter]
            y_cur = y[iter]
            forward_prop_1(X_cur, y_cur, params)
            if np.argmax(y_cur) != np.argmax(params["P2"]):
                error_counter += 1
            # print(np.argmax(y_cur), np.argmax(params["P2"]))
            # print(params["P2"])
        print(error_counter/lenX)
    

0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.65  0.55
0.6  0.5
0.55  0.5
0.35  0.35
0.3  0.25
0.3  0.25
0.25  0.25
0.25  0.25
0.25  0.25
0.25  0.25
0.25  0.25
0.25  0.25
0.25  0.25
0.25  0.25
0.25  0.25
0.25  0.25
0.25  0.25
0.25  0.25
0.25  0.25
0.25  0.25
0.25  0.25
0.25  0.25
0.25  0.25
0.25  0.25
0.25

In [21]:
error_counter = 0


for iter, item in enumerate(y):
    X_cur = X[iter % lenX]
    y_cur = y[iter % lenX]
    forward_prop_1(X_cur, y_cur, params)
    if np.argmax(y_cur) != np.argmax(params["P2"]):
        error_counter += 1
    # print(np.argmax(y_cur), np.argmax(params["P2"]))
    # print(params["P2"])
print(error_counter)



4
